<a href="https://colab.research.google.com/github/manojnsut/Water-Depth-Analyser/blob/main/Water_Depth_Analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

In [ ]:
label = pd.read_csv("/content/drive/MyDrive/Dataset/labels.csv", header=None, names=["label"])

In [ ]:
label.value_counts()

In [ ]:
label.columns

In [ ]:
def change(num):
    if(num <= 1):
        return 1
    elif(num > 1 and num < 2):
        return 2
    elif(num > 2 and num < 3):
        return 3
    elif(num > 3 and num < 4):
        return 4
    elif(num >= 4):
        return 5

label["label"] = label["label"].apply(lambda x : change(x))

In [ ]:
label.head()

In [ ]:
import os
import cv2

# Folder path containing the images
folder_path = '/content/drive/MyDrive/Dataset/dataset'

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith(('.jpg'))]

# Read and process each image
images = []
for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    image = cv2.imread(image_path)

    # Check if the image was successfully loaded
    if image is not None:
        images.append(image)
    else:
        print(f'Failed to load the image: {image_path}')


In [ ]:
images[0].shape

In [ ]:
numbers = np.array(label["label"])

In [ ]:
len(numbers)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((images, numbers))

In [ ]:
train_size = int(0.8 * len(dataset))  # 80% for training
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)


In [ ]:
batch_size = 32
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
sample = next(iter(train_dataset.take(1)))
image, label = sample[0], sample[1]
print("Image shape:", image.shape)
print("Label:", label)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
preprocess_input = tf.keras.applications.resnet.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), tf.one_hot(y, 4)))
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), tf.one_hot(y, 4)))


In [ ]:
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Add your custom head
num_classes = 4  # Number of output classes
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

def f1_score(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)
    return tf.keras.metrics.f1_score(y_true, y_pred, average='weighted')

# Define callback to calculate metrics at the end of each epoch
class MetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_predictions = self.model.predict(val_dataset)
        val_labels = tf.concat([y for _, y in val_dataset], axis=0)
        val_predictions = tf.argmax(val_predictions, axis=1)
        val_labels = tf.argmax(val_labels, axis=1)

        # Calculate F1 score
        f1 = classification_report(val_labels, val_predictions)
        print("F1 Score:\n", f1)

        # Calculate confusion matrix
        confusion = confusion_matrix(val_labels, val_predictions)
        print("Confusion Matrix:\n", confusion)


# Train the model
num_epochs = 10  # Number of epochs
history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Plot accuracy over epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
